# Sequence classification with Neural Networks
## Part 3: Basic CNN model

In [1]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

import altair as alt

import numpy as np
import pandas as pd

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from rnnprimer.datagen import generate_sample, Dataset

We're going to build a standard CNN architecture based on [this post](https://machinelearningmastery.com/cnn-models-for-human-activity-recognition-time-series-classification/) with two Conv1D layers and a single output layer for binary classification.

The learning rate is adjusted with a schedule for faster convergence given our simple data.

In [2]:
import tensorflow as tf
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        0.01,
        decay_steps=1000,
        decay_rate=0.5)

def get_cnn_model(n_features, n_timesteps):
    cnn_model = tf.keras.Sequential(
        [
            tf.keras.layers.Reshape((n_timesteps, n_features)),
            tf.keras.layers.Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(n_timesteps, n_features)),
            tf.keras.layers.Conv1D(filters=32, kernel_size=3, activation='relu'),
            tf.keras.layers.Dropout(0.5),
            tf.keras.layers.MaxPool1D(pool_size=1),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(20, activation='relu'),
            tf.keras.layers.Dense(1, activation="sigmoid")
        ]
    )

    cnn_model.compile(
            loss="binary_crossentropy",
            optimizer=tf.keras.optimizers.RMSprop(learning_rate=lr_schedule),
            metrics=[tf.keras.metrics.BinaryAccuracy()]
    )
    return cnn_model

In [3]:
data_cnn = []
n_timesteps=5

for outlier_prob in (0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0):
    print(outlier_prob)
    dataset = Dataset.generate(train_outlier_prob=outlier_prob, n_samples=100)
    model = get_cnn_model(n_features=1, n_timesteps=n_timesteps)

    model.fit(
        x=dataset.to_cnn_tfds(n_timesteps, batch_size=200),
        epochs=10,
        verbose=0
    )
    dataset = Dataset.generate(train_outlier_prob=outlier_prob, n_samples=20)
    res = model.evaluate(dataset.to_cnn_tfds(n_timesteps), verbose=0)
    data_cnn.append({'outlier_prob': outlier_prob, 'accuracy': res[1], 'window_size': n_timesteps})
    
df_cnn = pd.DataFrame(data_cnn)

0.01
0.05
0.1
0.2
0.3
0.4
0.5
0.6
0.7
0.8
0.9
1.0


In [4]:
alt.Chart(df_cnn[(df_cnn['window_size']==5)]).mark_line().encode(x='outlier_prob', y='accuracy')

alt.Chart(...)

CNN performance looks pretty good too, with a faster than RNN decline with high outlier probabilities. Performance here may also depend on the number of previous observations we give to the network. The fewer previous observations there are, the higher the chance that all of them would be outliers. So let's see what effect has the **window size** on the network performance.

In [ ]:
data_cnn = []
n_timesteps=10
for outlier_prob in (0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0):
    print(outlier_prob)
    dataset = Dataset.generate(train_outlier_prob=outlier_prob, n_samples=100)
    model = get_cnn_model(n_features=1, n_timesteps=n_timesteps)

    model.fit(
        x=dataset.to_cnn_tfds(n_timesteps, batch_size=200),
        epochs=10,
        verbose=0
    )
    dataset = Dataset.generate(train_outlier_prob=outlier_prob, n_samples=20)
    res = model.evaluate(dataset.to_cnn_tfds(n_timesteps), verbose=0)
    data_cnn.append({'outlier_prob': outlier_prob, 'accuracy': res[1], 'window_size': n_timesteps})

df_cnn = df_cnn.append(pd.DataFrame(data_cnn))

In [9]:
alt.Chart(df_cnn).mark_line().encode(x='outlier_prob', y='accuracy', color='window_size:N')

alt.Chart(...)

Our hypothesis confirmed, we can see that increasing the window size helps the model to perform better at higher levels of outliers.

Finally, as for RNN, let's see how the tensorboard graphs look like for CNN:

In [ ]:
# Clear any logs from previous runs
from datetime import datetime
!rm -rf ./logs/
log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

dataset = Dataset.generate(n_samples=200)

get_cnn_model(n_features=1, n_timesteps=5).fit(
    x=dataset.to_cnn_tfds(n_timesteps, batch_size=200),
    epochs=10,
    callbacks=[tensorboard_callback]
)

%tensorboard --logdir logs/fit